In [1]:
# %%
# Import base packages
import numpy as np
import argparse
import json
import logging
import os
import sys
from PIL import Image

# Import AWS training package
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3

In [7]:
curr_dir = os.getcwd()
path_raw = os.path.abspath(os.path.join(curr_dir, os.pardir, os.pardir, 'data', 'processed'))

local_path_train = os.path.join(path_raw, 'train')
local_path_val = os.path.join(path_raw, 'validation')
local_path_train_images = os.path.join(local_path_train, 'image')
local_path_train_annos = os.path.join(local_path_train, 'annos')
local_path_val_images = os.path.join(local_path_val, 'image')
local_path_val_annos = os.path.join(local_path_val, 'annos')

train_image_list = os.listdir(local_path_train_images)
train_annos_list = os.listdir(local_path_train_annos)
val_image_list = os.listdir(local_path_val_images)
val_annos_list = os.listdir(local_path_val_annos)

file_ids_train = [i.split('.')[0] for i in train_image_list]
file_ids_val = [i.split('.')[0] for i in val_image_list]

In [3]:
# %%
# Set the file paths
curr_dir = os.getcwd()
path_sample = os.path.abspath(os.path.join(curr_dir, os.pardir, 'data', 'sample_data'))
print(path_sample)

# Set up sagemaker session
sagemaker_session = sagemaker.Session(default_bucket = 'rohithdesikan-deepfashion')

# Get default bucket
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)

# Get role
role = sagemaker.get_execution_role()
print(role)

# set prefix, a descriptive name for the S3 directory
prefix = 'deepfashion_sample'

# upload all data to S3
# sagemaker_session.upload_data(path_sample, bucket=bucket, key_prefix=prefix)

# Set up file paths on S3
bucket_path = 's3://rohithdesikan-deepfashion/deepfashion-sample'
train_dir = 's3://rohithdesikan-deepfashion/deepfashion-sample/train'
val_dir = 's3://rohithdesikan-deepfashion/deepfashion-sample/val'
train_images_path = 's3://rohithdesikan-deepfashion/deepfashion-sample/train/image'
train_targets_path = 's3://rohithdesikan-deepfashion/deepfashion-sample/train/annos'
val_images_path = 's3://rohithdesikan-deepfashion/deepfashion-sample/val/image'
val_targets_path = 's3://rohithdesikan-deepfashion/deepfashion-sample/val/annos'
output_location = 's3://rohithdesikan-deepfashion/outputs'


/Users/rohithdesikan/Desktop/data_analysis/CV/deep_fashion/deepfashion/data/sample_data
rohithdesikan-deepfashion
arn:aws:iam::298061520854:role/rohithdesikan


In [6]:
train_image_list

['000026.jpg',
 '000032.jpg',
 '000033.jpg',
 '000027.jpg',
 '000019.jpg',
 '000031.jpg',
 '000025.jpg',
 '000024.jpg',
 '000030.jpg',
 '000018.jpg',
 '000034.jpg',
 '000020.jpg',
 '000008.jpg',
 '000009.jpg',
 '000021.jpg',
 '000035.jpg',
 '000023.jpg',
 '000037.jpg',
 '000036.jpg',
 '000022.jpg',
 '000079.jpg',
 '000045.jpg',
 '000051.jpg',
 '000086.jpg',
 '000092.jpg',
 '000093.jpg',
 '000087.jpg',
 '000050.jpg',
 '000044.jpg',
 '000078.jpg',
 '000052.jpg',
 '000046.jpg',
 '000091.jpg',
 '000085.jpg',
 '000084.jpg',
 '000090.jpg',
 '000047.jpg',
 '000053.jpg',
 '000057.jpg',
 '000043.jpg',
 '000094.jpg',
 '000080.jpg',
 '000081.jpg',
 '000095.jpg',
 '000042.jpg',
 '000056.jpg',
 '000040.jpg',
 '000054.jpg',
 '000068.jpg',
 '000083.jpg',
 '000097.jpg',
 '000096.jpg',
 '000082.jpg',
 '000069.jpg',
 '000055.jpg',
 '000041.jpg',
 '000058.jpg',
 '000064.jpg',
 '000070.jpg',
 '000071.jpg',
 '000065.jpg',
 '000059.jpg',
 '000073.jpg',
 '000067.jpg',
 '000098.jpg',
 '000099.jpg',
 '000066.j

In [ ]:
"source-ref": "s3://jsimon-groundtruth-demo/SSDB00001.JPG",
"GroundTruthDemo": {
  "annotations": [
    {"class_id": 0, "width": 54, "top": 482, "height": 39, "left": 337},
    {"class_id": 0, "width": 69, "top": 495, "height": 53, "left": 461},
    {"class_id": 0, "width": 52, "top": 482, "height": 41, "left": 523},
    {"class_id": 0, "width": 71, "top": 481, "height": 62, "left": 589},
    {"class_id": 0, "width": 347, "top": 479, "height": 120, "left": 573}
  ],
  "image_size": [{"width": 1280, "depth": 3, "height": 960}
]
},
"GroundTruthDemo-metadata": {
  "job-name": "labeling-job/groundtruthdemo",
  "class-map": {"0": "Car"},
  "human-annotated": "yes",
  "objects": [
    {"confidence": 0.94},
    {"confidence": 0.94},
    {"confidence": 0.94},
    {"confidence": 0.94},
    {"confidence": 0.94}
  ],
  "creation-date": "2018-11-26T04:01:09.038134",
  "type": "groundtruth/object-detection"
  }
}

In [ ]:
# %%
manifest_json = []
for file_id in file_ids_train:
    manifest_dict = {'source-ref' = train_images_path + rf"/{file_id}.jpg"}


In [ ]:
# %%
# Create image path and read in image
path_image = os.path.join(path_train_images, f'{trial_image}.jpg')
image = Image.open(path_image)
width, height = image.size
arr = np.array(image)
plt.imshow(image)

# Create annotations path and read it in
path_annos = os.path.join(path_train_annos, f'{trial_image}.json')
with open(path_annos,'r') as f:
    data = json.load(f)

num_items = len([k for k in list(data.keys()) if 'item' in k])

for i in range(1,num_items+1):
    label = data[f'item{i}']['category_id']
    x0,y0,x1,y1 = data[f'item{i}']['bounding_box']

    draw = ImageDraw.Draw(image)
    draw.rectangle([(x0, y0), (x1, y1)], outline='red')

plt.imshow(image)
